In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import zipfile as zp
import pickle
from sklearn.metrics import r2_score
from scikeras.wrappers import KerasRegressor
import os
import sklearn as sk
import scipy

In [19]:
# Print the version of numpy sklearn and tensorflow
print("Numpy version: ", np.__version__)
print("Pandas version: ", pd.__version__)
print("Tensorflow version: ", tf.__version__)
print(f" Sklearn version: {sk.__version__}")
print(f" Scipy version: {scipy.__version__}")

Numpy version:  1.23.5
Pandas version:  1.5.1
Tensorflow version:  2.12.0
 Sklearn version: 1.2.1
 Scipy version: 1.10.0


In [20]:
#Open the zip file in context manager
with zp.ZipFile('Data.zip', 'r') as myzip:
    #List the files in the zip file
    # myzip.printdir()
    # Save printdir to a variable
    files = myzip.namelist()
files_sorted = sorted(files[3:])
files_from_1970 = files_sorted[550:]

In [21]:
with open('intersection.pkl', 'rb') as f:
    intersection = pickle.load(f)
intersection.append('DATE')

In [23]:
def get_data_to_matrix(intersection, files_from_1970):
    z = np.zeros((1, 70))
    x = np.zeros((1, 70))
    returns = np.array([])
    number_of_obs = {}
    for i, file in enumerate(files_from_1970):
        full_name = 'Data/' + file
        df = pd.read_csv(full_name)
        df.drop(columns=['Unnamed: 0', 'mve0', 'prc', 'SHROUT', 'sic2'], inplace=True)
        df = df[intersection]
        df.dropna(axis=0, thresh=60, inplace=True)
        # df.drop(columns=['RET'], inplace=True)
        df = df.apply(lambda x: (x - x.mean()) / x.std() if (x.name not in ('permno','RET','DATE'))  else x)
        df.fillna(0, inplace=True)
        Z = df.drop(columns=['permno', 'RET', 'DATE'])
        Z = Z.to_numpy()
        X = np.linalg.inv(Z.T @ Z) @ Z.T @ df['RET'].to_numpy()
        X_stacked = np.tile(X, (len(Z), 1))
        ret = df['RET'].to_numpy()
        number_of_obs[file] = (len(ret))
        z = np.vstack((z, Z))
        x = np.vstack((x, X_stacked))
        returns = np.append(returns, ret)
        print(f"File {file} read successfully.")
    z = z[1:]
    x = x[1:]
    return z, x, returns, number_of_obs

In [2]:
reload_files = False
if reload_files == True:
    Z, X, returns, number_of_obs = get_data_to_matrix(intersection, files_from_1970)

    with open('characters.pkl', 'wb') as f:
        pickle.dump(Z, f)
    with open('factors.pkl', 'wb') as f:
        pickle.dump(X, f)
    with open('returns.pkl', 'wb') as f:
        pickle.dump(returns, f)
    with open('number_of_obs.pkl', 'wb') as f:
        pickle.dump(number_of_obs, f)
else:
    with open('characters.pkl', 'rb') as f:
        Z = pickle.load(f)
    with open('factors.pkl', 'rb') as f:
        X = pickle.load(f)
    with open('returns.pkl', 'rb') as f:
        returns = pickle.load(f)
    with open('number_of_obs.pkl', 'rb') as f:
        number_of_obs = pickle.load(f)

In [6]:
# Sum the values in number_of_obs until we get to 2005 in the keys
sum_train = 0
for key, value in number_of_obs.items():
    if '20050129' in key:
        print(sum_train)
        break
    sum_train += value

sum_val = 0
for key, value in number_of_obs.items():
    if '20100129' in key:
        print(sum_val)
        break
    sum_val += value

2026950


In [3]:

z_test = Z[2026950:]
x_test = X[2026950:]
returns_test = returns[2026950:]
z_x_test = np.hstack((z_test, x_test))

z_x_train = np.concatenate((Z[:1737557], X[:1737557]), axis=1)
z_x_val = np.concatenate((Z[1737557:2026950], X[1737557:2026950]), axis=1)

In [4]:
from tensorflow import keras
from keras.layers import Dense
from keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor='val_loss', patience=3)
def create_one_hidden_layer_mod(no_units_activ, no_factors, activ, l_rate, inp_shape, reg, reg_type, kernel_init, optimizer):
    inputs = keras.Input(shape=(inp_shape,), name='Input')

    layer_input_dim = inp_shape // 2
    inputs_1 = inputs[:,:70]

    inputs_2 = inputs[:,70:]
    # Split the two inputs into two different layers
    print(inputs_2)
    x_1 = Dense(units=no_units_activ,
        input_shape=(layer_input_dim,),
        activation=activ,
        use_bias=True,
        kernel_initializer=kernel_init,
        kernel_regularizer=reg_type(reg))(inputs_1)
    x_2_out = Dense(units=no_factors,
            input_shape=(layer_input_dim,),
            activation='linear',
            use_bias=True,
            kernel_initializer=kernel_init,
            kernel_regularizer=reg_type(reg))(inputs_2)
    x_1_out = Dense(units=no_factors,
            # input_shape=(inp_shape,),
            activation='linear',
            use_bias=True,
            kernel_initializer=kernel_init,
            kernel_regularizer=reg_type(reg)
            )(x_1)
    #The output will be the dot product of the two hidden layers
    outputs = keras.layers.Dot(axes=1)([x_1_out, x_2_out])

    model = keras.Model(inputs=inputs, outputs=outputs, name='autoencoder')
    model.compile(
        optimizer=optimizer(learning_rate=l_rate),
        loss='mean_squared_error')
    return model

In [64]:
def create_two_hidden_layer_network(no_units_first, no_units_second, no_factors, activ, l_rate,  inp_shape, reg, reg_type, kernel_init):
    inputs = keras.Input(shape=(inp_shape,), name='Input')
    # print(inputs)
    layer_input_dim = inp_shape // 2
    inputs_1 = inputs[:,:70]
    # print(inputs_1)
    inputs_2 = inputs[:,70:]
    x_1 = Dense(units=no_units_first,
        input_shape=(layer_input_dim,),
        activation=activ,
        use_bias=True,
        kernel_initializer=kernel_init,
        kernel_regularizer=reg_type(reg))(inputs_1)
    x_1_hidden = Dense(units=no_units_second,
        activation=activ,
        use_bias=True,
        kernel_initializer=kernel_init,
        kernel_regularizer=reg_type(reg))(x_1)
    x_2_out = Dense(units=no_factors,
            input_shape=(layer_input_dim,),
            activation='linear',
            use_bias=True,
            kernel_initializer=kernel_init,
            kernel_regularizer=reg_type(reg))(inputs_2)
    x_1_out = Dense(units=no_factors,
            activation='linear',
            use_bias=True,
            kernel_initializer=kernel_init,
            kernel_regularizer=reg_type(reg)
            # kernel_regularizer=keras.regularizers.L1(0.001),
            )(x_1_hidden)
    #The output will be the dot product of the two hidden layers
    outputs = keras.layers.Dot(axes=1)([x_1_out, x_2_out])

    model = keras.Model(inputs=inputs, outputs=outputs, name='autoencoder')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=l_rate),
        loss='mean_squared_error')
    return model

In [6]:
def create_three_hidden_layer_network(no_units_first, no_units_second, no_units_third, no_factors, activ, l_rate, inp_shape, reg, reg_type, kernel_init):
    inputs = keras.Input(shape=(inp_shape,), name='Input')
    # print(inputs)
    layer_input_dim = inp_shape // 2
    inputs_1 = inputs[:,:70]
    # print(inputs_1)
    inputs_2 = inputs[:,70:]
    x_1 = Dense(units=no_units_first,
        input_shape=(layer_input_dim,),
        activation=activ,
        use_bias=True,
        kernel_initializer=kernel_init,
        kernel_regularizer=reg_type(reg))(inputs_1)
    x_1_hidden = Dense(units=no_units_second,
        activation=activ,
        use_bias=True,
        kernel_initializer=kernel_init,
        kernel_regularizer=reg_type(reg))(x_1)
    x_1_hidden_2 = Dense(units=no_units_third,
        activation=activ,
        use_bias=True,
        kernel_initializer=kernel_init,
        kernel_regularizer=reg_type(reg))(x_1_hidden)
    x_2_out = Dense(units=no_factors,
            input_shape=(layer_input_dim,),
            activation='linear',
            use_bias=True,
            kernel_initializer=kernel_init,
            kernel_regularizer=reg_type(reg))(inputs_2)
    x_1_out = Dense(units=no_factors,
            activation='linear',
            use_bias=True,
            kernel_initializer=kernel_init,
            kernel_regularizer=keras.regularizers.L1(reg)
            # kernel_regularizer=keras.regularizers.L1(0.001),
            )(x_1_hidden_2)
    #The output will be the dot product of the two hidden layers
    outputs = keras.layers.Dot(axes=1)([x_1_out, x_2_out])

    model = keras.Model(inputs=inputs, outputs=outputs, name='autoencoder')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=l_rate),
        loss='mean_squared_error')
    return model

3 Factor Model

In [9]:
neural_net_distributions_one_hh  = {
    'no_units_activ': [16, 32, 64],
    'activ': ['relu', 'sigmoid'],
    'reg_type': [keras.regularizers.l1, keras.regularizers.l2],
    'reg': [0, 0.001, 0.01],
    'l_rate': [0.0001, 0.001, 0.01],
    'kernel_init': ['he_uniform'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20],
    'inp_shape': [z_x_train.shape[1]],
    'no_factors': [3]}

neural_net_distributions_two_hh  = {
    'no_units_first': [16, 32, 64],
    'no_units_second': [16, 32, 64],
    'activ': ['relu', 'sigmoid'],
    'reg_type': [keras.regularizers.l1, keras.regularizers.l2],
    'reg': [0, 0.001, 0.01],
    'l_rate': [0.0001, 0.001, 0.01],
    'kernel_init': ['he_uniform'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 50],
    'inp_shape': [z_x_train.shape[1]],
    'no_factors': [3]}

neural_net_distributions_three_hh  = {
    'no_units_first': [16, 32, 64],
    'no_units_second': [16, 32, 64],
    'no_units_third': [16, 32, 64],
    'activ': ['relu', 'sigmoid'],
    'reg_type': [keras.regularizers.l1, keras.regularizers.l2],
    'reg': [0, 0.001, 0.01],
    'l_rate': [0.0001, 0.001, 0.01],
    'kernel_init': ['he_uniform'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 50],
    'inp_shape': [z_x_train.shape[1]],
    'no_factors': [3]}

In [7]:
# Use GridSearchCV to find the best parameters
from keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor='val_loss', patience=3)
from keras.wrappers.scikit_learn import KerasRegressor
# import randint, unfirom from scipy.stats
from scipy.stats import randint, uniform

# Baseline Estimation

In [ ]:
z_train = Z[:1737557]
x_train = X[:1737557]
returns_train = returns[:1737557]
# x_train = np.hstack((z_train, x_train))

z_val = Z[1737557:2026950]
x_val = X[1737557:2026950]
returns_val = returns[1737557:2026950]
# x_val = np.hstack((z_val, x_val))

# z_test = Z[2026950:]
# x_test = X[2026950:]
# returns_test = returns[2026950:]
# x_test = np.hstack((z_test, x_test))

z_x_train = np.concatenate((Z[:1737557], X[:1737557]), axis=1)
z_x_val = np.concatenate((Z[1737557:2026950], X[1737557:2026950]), axis=1)

In [ ]:
from scikeras.wrappers import KerasRegressor

In [ ]:
from tensorflow import keras
from keras.layers import Dense
from keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor='val_loss', patience=3)
def create_one_hidden_layer_mod(no_units_activ,no_factors, activ, l_rate, inp_shape, reg):
    inputs = keras.Input(shape=(inp_shape,), name='Input')
    # print(inputs)
    layer_input_dim = inp_shape // 2
    inputs_1 = inputs[:,:70]
    # print(inputs_1)
    inputs_2 = inputs[:,70:]
    print(inputs_2)
    x_1 = Dense(units=no_units_activ,
        input_shape=(layer_input_dim,),
        activation=activ,
        use_bias=True,
        kernel_initializer='he_uniform',
        kernel_regularizer=keras.regularizers.L1(reg))(inputs_1)
    x_2_out = Dense(units=no_factors,
            input_shape=(layer_input_dim,),
            activation='linear',
            use_bias=True,
            kernel_initializer='he_uniform',
            kernel_regularizer=keras.regularizers.L1(reg))(inputs_2)
    x_1_out = Dense(units=no_factors,
            # input_shape=(inp_shape,),
            activation='linear',
            use_bias=True,
            kernel_initializer='he_uniform',
            kernel_regularizer=keras.regularizers.L1(reg)
            )(x_1)
    #The output will be the dot product of the two hidden layers
    outputs = keras.layers.Dot(axes=1)([x_1_out, x_2_out])

    model = keras.Model(inputs=inputs, outputs=outputs, name='autoencoder')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=l_rate),
        loss='mean_squared_error')
    return model

In [ ]:
model = create_one_hidden_layer_mod(no_units_activ=32, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)

KerasTensor(type_spec=TensorSpec(shape=(None, 140), dtype=tf.float32, name='Input'), name='Input', description="created by layer 'Input'")
KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_4/strided_slice:0', description="created by layer 'tf.__operators__.getitem_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_5/strided_slice:0', description="created by layer 'tf.__operators__.getitem_5'")


In [ ]:
history = model.fit(z_x_train, returns_train, epochs=20, batch_size=10000, validation_data=(z_x_val, returns_val), callbacks=[earlystop])


Epoch 1/20
174/174 [==============================] - 5s 17ms/step - loss: 0.0319 - val_loss: 0.0293
Epoch 2/20
174/174 [==============================] - 2s 13ms/step - loss: 0.0303 - val_loss: 0.0287
Epoch 3/20
174/174 [==============================] - 2s 13ms/step - loss: 0.0299 - val_loss: 0.0287
Epoch 4/20
174/174 [==============================] - 2s 12ms/step - loss: 0.0298 - val_loss: 0.0286
Epoch 5/20
174/174 [==============================] - 2s 12ms/step - loss: 0.0297 - val_loss: 0.0286
Epoch 6/20
174/174 [==============================] - 2s 12ms/step - loss: 0.0296 - val_loss: 0.0287
Epoch 7/20
174/174 [==============================] - 2s 12ms/step - loss: 0.0296 - val_loss: 0.0286
Epoch 8/20
174/174 [==============================] - 2s 12ms/step - loss: 0.0295 - val_loss: 0.0287


In [ ]:
# Use GridSearchCV to find the best parameters
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor='val_loss', patience=3)
model = KerasRegressor(build_fn=create_one_hidden_layer_mod, verbose=1)
# define the grid search parameters
no_units_activs = [16, 32, 64]
# no_factors = [3, 5, 7]
activs = ['relu']
# regularizers = [0, 0.001, 0.01]
# learning_rates = [0.0002, 0.002, 0.001]
batch_sizes = [50, 100, 500, 1000]
epochs = [20, 50, 100]
# earlystops = [EarlyStopping(monitor='val_loss', patience=3),EarlyStopping(monitor='val_loss', patience=5)]

param_grid = dict(no_units_activ=no_units_activs, no_factors=[3], activ=activs,
                  inp_shape=[z_x_train.shape[1]], reg=[0], l_rate=[0.002], batch_size=batch_sizes, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=3, cv=3)
grid_result = grid.fit(z_x_train, returns[:1737557], validation_data=(z_x_val, returns[1737557:2026950]), callbacks=[earlystop])

C:\Users\berta\AppData\Local\Temp\ipykernel_34624\1500825809.py:6: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_one_hidden_layer_mod, verbose=1)


KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_1/strided_slice:0', description="created by layer 'tf.__operators__.getitem_1'")
Epoch 1/50
1738/1738 [==============================] - 4s 2ms/step - loss: 0.0305 - val_loss: 0.0289
Epoch 2/50
1738/1738 [==============================] - 2s 1ms/step - loss: 0.0298 - val_loss: 0.0286
Epoch 3/50
1738/1738 [==============================] - 2s 1ms/step - loss: 0.0297 - val_loss: 0.0285
Epoch 4/50
1738/1738 [==============================] - 2s 1ms/step - loss: 0.0296 - val_loss: 0.0286
Epoch 5/50
1738/1738 [==============================] - 2s 1ms/step - loss: 0.0296 - val_loss: 0.0286
Epoch 6/50
1738/1738 [==============================] - 3s 1ms/step - loss: 0.0296 - val_loss: 0.0286


In [ ]:
grid_result.best_params_

{'activ': 'relu',
 'batch_size': 1000,
 'epochs': 50,
 'inp_shape': 140,
 'l_rate': 0.002,
 'no_factors': 3,
 'no_units_activ': 16,
 'reg': 0}

In [ ]:
model = create_one_hidden_layer_mod(no_units_activ=32, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)

KerasTensor(type_spec=TensorSpec(shape=(None, 140), dtype=tf.float32, name='Input'), name='Input', description="created by layer 'Input'")
KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_2/strided_slice:0', description="created by layer 'tf.__operators__.getitem_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_3/strided_slice:0', description="created by layer 'tf.__operators__.getitem_3'")


In [ ]:
BATCH_SIZE = 1000
EPOCHS = 20
history = model.fit(
    x=z_x_train,
    y=returns[:1737557],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_val, returns[1737557:2026950]),
    callbacks=[earlystop])

Epoch 1/20
1738/1738 [==============================] - 8s 4ms/step - loss: 0.0304 - val_loss: 0.0288
Epoch 2/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0298 - val_loss: 0.0286
Epoch 3/20
1738/1738 [==============================] - 6s 4ms/step - loss: 0.0296 - val_loss: 0.0285
Epoch 4/20
1738/1738 [==============================] - 6s 4ms/step - loss: 0.0296 - val_loss: 0.0286
Epoch 5/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0295 - val_loss: 0.0284
Epoch 6/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0295 - val_loss: 0.0286
Epoch 7/20
1738/1738 [==============================] - 6s 4ms/step - loss: 0.0294 - val_loss: 0.0285
Epoch 8/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0294 - val_loss: 0.0285


In [ ]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_val)


#Calculate the R-squared of the model
from sklearn.metrics import r2_score
print(f"Out of Sample R^2: {r2_score(returns[1737557:2026950], pred)}")

9044/9044 [==============================] - 15s 2ms/step
Out of Sample R^2: 0.12401829972240086


#### 2 Hidden Layers

In [ ]:
def create_two_hidden_layer_network(no_units_first, no_units_second,no_factors, activ, l_rate, inp_shape, reg):
    inputs = keras.Input(shape=(inp_shape,), name='Input')
    # print(inputs)
    layer_input_dim = inp_shape // 2
    inputs_1 = inputs[:,:70]
    # print(inputs_1)
    inputs_2 = inputs[:,70:]
    x_1 = Dense(units=no_units_first,
        input_shape=(layer_input_dim,),
        activation=activ,
        use_bias=True,
        kernel_initializer='he_uniform',
        kernel_regularizer=keras.regularizers.L1(reg))(inputs_1)
    x_1_hidden = Dense(units=no_units_second,
        activation=activ,
        use_bias=True,
        kernel_initializer='he_uniform',
        kernel_regularizer=keras.regularizers.L1(reg))(x_1)
    x_2_out = Dense(units=no_factors,
            input_shape=(layer_input_dim,),
            activation='linear',
            use_bias=True,
            kernel_initializer='he_uniform',
            kernel_regularizer=keras.regularizers.L1(reg))(inputs_2)
    x_1_out = Dense(units=no_factors,
            activation='linear',
            use_bias=True,
            kernel_initializer='he_uniform',
            kernel_regularizer=keras.regularizers.L1(reg)
            # kernel_regularizer=keras.regularizers.L1(0.001),
            )(x_1_hidden)
    #The output will be the dot product of the two hidden layers
    outputs = keras.layers.Dot(axes=1)([x_1_out, x_2_out])

    model = keras.Model(inputs=inputs, outputs=outputs, name='autoencoder')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=l_rate),
        loss='mean_squared_error')
    return model

In [ ]:
model_2 = create_two_hidden_layer_network(no_units_first=32, no_units_second=16, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)

In [ ]:
history = model_2.fit(z_x_train, returns[:1737557], epochs=20, batch_size=1000, validation_data=(z_x_val, returns[1737557:2026950]), callbacks=[earlystop])

Epoch 1/20
1738/1738 [==============================] - 10s 5ms/step - loss: 0.0303 - val_loss: 0.0288
Epoch 2/20
1738/1738 [==============================] - 8s 5ms/step - loss: 0.0297 - val_loss: 0.0288
Epoch 3/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0296 - val_loss: 0.0286
Epoch 4/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0295 - val_loss: 0.0286
Epoch 5/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0295 - val_loss: 0.0287
Epoch 6/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0294 - val_loss: 0.0285
Epoch 7/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0293 - val_loss: 0.0291
Epoch 8/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0293 - val_loss: 0.0285
Epoch 9/20
1738/1738 [==============================] - 7s 4ms/step - loss: 0.0293 - val_loss: 0.0287


In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV
model_2 = KerasRegressor(build_fn=create_two_hidden_layer_network)
# define the grid search parameters
no_first = [8, 16, 32, 64, 128]
no_second = [8, 16, 32, 64, 128]
# no_factors = [3, 5, 7]
# activs = ['relu', 'gelu']
activs = ['relu']
# regularizers = [0, 0.001, 0.01]
# learning_rates = [0.0002, 0.002, 0.001]
# batch_sizes = [1000, 5000,10000, 20000]
epochs = [50]

param_grid_2 = dict(no_units_first=no_first, no_units_second=no_second,
                     no_factors=[3],activ=activs, inp_shape=[z_x_train.shape[1]],
                       reg=[0], l_rate=[0.002], batch_size=[1000])
grid_2 = GridSearchCV(estimator=model_2, param_grid=param_grid_2,n_jobs=3, cv=3)
grid_result_2 = grid_2.fit(z_x_train, returns[:1737557], validation_data=(z_x_val, returns[1737557:2026950]), callbacks=[earlystop])

C:\Users\berta\AppData\Local\Temp\ipykernel_1732\99865868.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_2 = KerasRegressor(build_fn=create_two_hidden_layer_network)


1738/1738 [==============================] - 13s 6ms/step - loss: 0.0303 - val_loss: 0.0288


In [ ]:
grid_result_2.best_params_

{'activ': 'relu',
 'batch_size': 1000,
 'inp_shape': 140,
 'l_rate': 0.002,
 'no_factors': 3,
 'no_units_first': 64,
 'no_units_second': 32,
 'reg': 0}

#### 3 Hidden Layers

In [ ]:
def create_three_hidden_layer_network(no_units_first, no_units_second, no_units_third, no_factors, activ, l_rate, inp_shape, reg):
    inputs = keras.Input(shape=(inp_shape,), name='Input')
    # print(inputs)
    layer_input_dim = inp_shape // 2
    inputs_1 = inputs[:,:70]
    # print(inputs_1)
    inputs_2 = inputs[:,70:]
    x_1 = Dense(units=no_units_first,
        input_shape=(layer_input_dim,),
        activation=activ,
        use_bias=True,
        kernel_initializer='he_uniform',
        kernel_regularizer=keras.regularizers.L1(reg))(inputs_1)
    x_1_hidden = Dense(units=no_units_second,
        activation=activ,
        use_bias=True,
        kernel_initializer='he_uniform',
        kernel_regularizer=keras.regularizers.L1(reg))(x_1)
    x_1_hidden_2 = Dense(units=no_units_third,
        activation=activ,
        use_bias=True,
        kernel_initializer='he_uniform',
        kernel_regularizer=keras.regularizers.L1(reg))(x_1_hidden)
    x_2_out = Dense(units=no_factors,
            input_shape=(layer_input_dim,),
            activation='linear',
            use_bias=True,
            kernel_initializer='he_uniform',
            kernel_regularizer=keras.regularizers.L1(reg))(inputs_2)
    x_1_out = Dense(units=no_factors,
            activation='linear',
            use_bias=True,
            kernel_initializer='he_uniform',
            kernel_regularizer=keras.regularizers.L1(reg)
            # kernel_regularizer=keras.regularizers.L1(0.001),
            )(x_1_hidden_2)
    #The output will be the dot product of the two hidden layers
    outputs = keras.layers.Dot(axes=1)([x_1_out, x_2_out])

    model = keras.Model(inputs=inputs, outputs=outputs, name='autoencoder')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=l_rate),
        loss='mean_squared_error')
    return model

Here we Restrict the Model parameters although before this final version we have tried grid-search for differen batch-sizes, learning rates and activation functions

In [ ]:

model_3 = KerasRegressor(build_fn=create_three_hidden_layer_network)
# define the grid search parameters
no_first = [8, 16, 32, 64, 128]
no_second = [8, 16, 32, 64, 128]
no_third = [8, 16, 32, 64, 128]
# no_factors = [3, 5, 7]
# activs = ['relu', 'gelu']
# regularizers = [0, 0.001, 0.01]
# learning_rates = [0.0002, 0.002, 0.001]
# batch_sizes = [1000, 5000,10000, 20000]

param_grid_3 = dict(no_units_first=no_first, no_units_second=no_second,
                    no_units_third=no_third, no_factors=[3],activ=['relu'],
                    l_rate=[0.002], inp_shape=[z_x_train.shape[1]],
                    reg=[0], batch_size=[1000])
grid_3 = GridSearchCV(estimator=model_3, param_grid=param_grid_3,n_jobs=3, cv=3)
grid_result_3 = grid_3.fit(z_x_train, returns[:1737557], validation_data=(z_x_val, returns[1737557:2026950]), callbacks=[earlystop])

C:\Users\berta\AppData\Local\Temp\ipykernel_1732\2567158883.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_3 = KerasRegressor(build_fn=create_three_hidden_layer_network)


1738/1738 [==============================] - 14s 7ms/step - loss: 0.0304 - val_loss: 0.0283


In [ ]:
grid_result_3.best_params_

{'activ': 'relu',
 'batch_size': 1000,
 'inp_shape': 140,
 'l_rate': 0.002,
 'no_factors': 3,
 'no_units_first': 64,
 'no_units_second': 128,
 'no_units_third': 32,
 'reg': 0}

## Evaluate the Model at optimal Hyperparameters

In [ ]:
z_x_train = np.concatenate((Z[:1737557], X[:1737557]), axis=1)
z_x_val = np.concatenate((Z[1737557:2026950], X[1737557:2026950]), axis=1)

In [ ]:
def create_total_r2(actual_returns, pred_returns):
    denom = np.sum(actual_returns**2)
    nom = np.sum((actual_returns - pred_returns)**2)
    r_2 = 1 - nom / denom
    return r_2

In [ ]:
z_x_test = np.concatenate((Z[2026950:], X[2026950:]), axis=1)

### 3 Factors

##### One Hidden Layer

In [ ]:
model = create_one_hidden_layer_mod(no_units_activ=16, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)
BATCH_SIZE = 1000
EPOCHS = 50
history = model.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:2026950],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns[2026950:]),
    callbacks=[earlystop])

KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_3/strided_slice:0', description="created by layer 'tf.__operators__.getitem_3'")
Epoch 1/50
2027/2027 [==============================] - 5s 2ms/step - loss: 0.0303 - val_loss: 0.0247
Epoch 2/50
2027/2027 [==============================] - 3s 1ms/step - loss: 0.0296 - val_loss: 0.0247
Epoch 3/50
2027/2027 [==============================] - 3s 1ms/step - loss: 0.0294 - val_loss: 0.0247
Epoch 4/50
2027/2027 [==============================] - 2s 1ms/step - loss: 0.0294 - val_loss: 0.0246
Epoch 5/50
2027/2027 [==============================] - 2s 1ms/step - loss: 0.0293 - val_loss: 0.0247
Epoch 6/50
2027/2027 [==============================] - 2s 1ms/step - loss: 0.0293 - val_loss: 0.0247
Epoch 7/50
2027/2027 [==============================] - 2s 1ms/step - loss: 0.0293 - val_loss: 0.0247


##### Out of Sample + Total $R^{2}$

In [ ]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_test)
in_samp = model.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns[2026950:], np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

Out of Sample R^2: 0.11180776707959461
Total R^2: 0.14398162478295573


In [ ]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_test)
in_samp = model.predict(np.concatenate([z_x_train, z_x_val]))


#Calculate the R-squared of the model
from sklearn.metrics import r2_score
print(f"Out of Sample R^2: {r2_score(returns[2026950:], pred)}")
print(f"Total R^2: {r2_score(returns, np.concatenate([in_samp, pred]))}")

63343/63343 [==============================] - 34s 536us/step
Out of Sample R^2: 0.09988282788810265
Total R^2: 0.1401665390353658


##### Predictive $R^{2}$

Here we need to get the weights corresponding to the factors to get the in Sample average and then use the weights corresponding to the betas to get the prediction.

In [ ]:
from keras import backend as K

In [ ]:
# This confirms we get the same output
lambda_facts = np.mean(z_x_train[:,70:] @ model.get_weight_paths()['dense_4.kernel'].numpy() + np.tile(model.get_weight_paths()['dense_4.bias'].numpy(),(z_x_train[:,70:].shape[0],1)),axis=0)

In [ ]:
get_5th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
lambda_new = get_5th_layer_output([z_x_train])[0]
lambda_facts = np.mean(lambda_new, axis=0)

In [ ]:
# This is just to confirm that we get the right output
weights_first = model.get_weight_paths()['dense_3.kernel'].numpy()
bias_first = model.get_weight_paths()['dense_3.bias'].numpy()
weights_second = model.get_weight_paths()['dense_5.kernel'].numpy()
bias_second = model.get_weight_paths()['dense_5.bias'].numpy()
np.maximum(z_x_test[:,:70] @ weights_first + bias_first,0) @ weights_second + bias_second

In [ ]:
# Get the output of the 5th layer
get_4th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[4].output])
betas = get_4th_layer_output(z_x_test)[0]

In [ ]:
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)

### Predictive R^{2}

In [ ]:
create_total_r2(returns[2026950:], predicted)

0.001571634861741722

##### Two Hidden Layers

In [ ]:
model_2 = create_two_hidden_layer_network(no_units_first=64, no_units_second=32, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)
BATCH_SIZE = 1000
EPOCHS = 50
history = model_2.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:2026950],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns[2026950:]),
    callbacks=[earlystop])

Epoch 1/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0300 - val_loss: 0.0248
Epoch 2/50
2027/2027 [==============================] - 5s 2ms/step - loss: 0.0294 - val_loss: 0.0247
Epoch 3/50
2027/2027 [==============================] - 5s 3ms/step - loss: 0.0293 - val_loss: 0.0247
Epoch 4/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0292 - val_loss: 0.0247
Epoch 5/50
2027/2027 [==============================] - 5s 2ms/step - loss: 0.0291 - val_loss: 0.0247
Epoch 6/50
2027/2027 [==============================] - 4s 2ms/step - loss: 0.0290 - val_loss: 0.0247


In [ ]:
#Use the model weights to predict returns on the validation set
pred = model_2.predict(z_x_test)
in_samp = model_2.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns[2026950:], np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

63343/63343 [==============================] - 36s 567us/step
Out of Sample R^2: 0.10936992490748976
Total R^2: 0.15454757183175816


##### Predictive R^{2}

In [ ]:
get_6th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[6].output])

lambda_new = get_6th_layer_output_2(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)


In [ ]:
get_5th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[5].output])
betas = get_5th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
create_total_r2(returns[2026950:], predicted)

0.0038198255401545866

In [ ]:
pred = model_2.predict(z_x_test)
in_samp = model_2.predict(np.concatenate([z_x_train, z_x_val]))


#Calculate the R-squared of the model
print(f"Out of Sample R^2: {r2_score(returns[2026950:], pred)}")
print(f"Total R^2: {r2_score(returns, np.concatenate([in_samp, pred]))}")

63343/63343 [==============================] - 36s 571us/step
Out of Sample R^2: 0.09550881572914593
Total R^2: 0.17655499946706432


##### Three Hidden Layers

In [ ]:
model_3 = create_three_hidden_layer_network(no_units_first=64, no_units_second=128, no_units_third=32, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)
BATCH_SIZE = 1000
EPOCHS = 50
history = model_3.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:2026950],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns[2026950:]),
    callbacks=[earlystop])

Epoch 1/50
2027/2027 [==============================] - 8s 3ms/step - loss: 0.0300 - val_loss: 0.0248
Epoch 2/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0295 - val_loss: 0.0247
Epoch 3/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0293 - val_loss: 0.0249
Epoch 4/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0292 - val_loss: 0.0247
Epoch 5/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0291 - val_loss: 0.0248
Epoch 6/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0290 - val_loss: 0.0248
Epoch 7/50
2027/2027 [==============================] - 6s 3ms/step - loss: 0.0289 - val_loss: 0.0248


In [ ]:
pred = model_3.predict(z_x_test)
in_samp = model_3.predict(np.concatenate([z_x_train, z_x_val]))


#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns[2026950:], np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

63343/63343 [==============================] - 36s 558us/step
Out of Sample R^2: 0.10232412097092813
Total R^2: 0.15815373634395635


Old version

In [ ]:
pred = model_3.predict(z_x_test)
in_samp = model_3.predict(np.concatenate([z_x_train, z_x_val]))

print(f"Out of Sample R^2: {r2_score(returns[2026950:], pred)}")
print(f"Total R^2: {r2_score(returns, np.concatenate([in_samp, pred]))}")

63343/63343 [==============================] - 36s 571us/step
Out of Sample R^2: 0.06729522696342505
Total R^2: 0.2038230349076794


##### Predictive R^{2}

In [ ]:
get_6th_layer_output_3 = K.function([model_3.layers[0].input],
                                  [model_3.layers[7].output])

lambda_new = get_6th_layer_output_3(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)

In [ ]:
get_7th_layer_output_2 = K.function([model_3.layers[0].input],
                                  [model_3.layers[6].output])
betas = get_7th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
create_total_r2(returns[2026950:], predicted)

0.003683647041954896

### 5 Factors

In [ ]:
model = create_one_hidden_layer_mod(no_units_activ=16, no_factors=5, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)
BATCH_SIZE = 1000
EPOCHS = 50
history = model.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:2026950],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns[2026950:]),
    callbacks=[earlystop])

KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_1/strided_slice:0', description="created by layer 'tf.__operators__.getitem_1'")
Epoch 1/50
2027/2027 [==============================] - 13s 5ms/step - loss: 0.0299 - val_loss: 0.0247
Epoch 2/50
2027/2027 [==============================] - 9s 4ms/step - loss: 0.0292 - val_loss: 0.0244
Epoch 3/50
2027/2027 [==============================] - 8s 4ms/step - loss: 0.0291 - val_loss: 0.0245
Epoch 4/50
2027/2027 [==============================] - 8s 4ms/step - loss: 0.0290 - val_loss: 0.0244
Epoch 5/50
2027/2027 [==============================] - 8s 4ms/step - loss: 0.0290 - val_loss: 0.0244
Epoch 6/50
2027/2027 [==============================] - 7s 4ms/step - loss: 0.0289 - val_loss: 0.0244
Epoch 7/50
2027/2027 [==============================] - 8s 4ms/step - loss: 0.0289 - val_loss: 0.0244
Epoch 8/50
2027/2027 [==============================] - 8s 4ms/step - loss: 0.0289 - val_los

In [ ]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_test)
in_samp = model.predict(np.concatenate([z_x_train, z_x_val]))

#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns[2026950:], np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

Out of Sample R^2: 0.11836962257042472
Total R^2: 0.1542018069355764


##### Predictive $R^{2}$

In [ ]:
get_5th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
lambda_new = get_5th_layer_output([z_x_train])[0]
lambda_facts = np.mean(lambda_new, axis=0)
# Get the output of the 5th layer
get_4th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[4].output])
betas = get_4th_layer_output(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
create_total_r2(returns[2026950:], predicted)

-0.0012065008257415855

##### 2 Hidden Layers

In [ ]:
model_2 = create_two_hidden_layer_network(no_units_first=64, no_units_second=32, no_factors=5, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)
BATCH_SIZE = 1000
EPOCHS = 50
history = model_2.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:2026950],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns[2026950:]),
    callbacks=[earlystop])

Epoch 1/50
2027/2027 [==============================] - 16s 7ms/step - loss: 0.0298 - val_loss: 0.0246
Epoch 2/50
2027/2027 [==============================] - 13s 6ms/step - loss: 0.0291 - val_loss: 0.0245
Epoch 3/50
2027/2027 [==============================] - 13s 6ms/step - loss: 0.0289 - val_loss: 0.0245
Epoch 4/50
2027/2027 [==============================] - 13s 6ms/step - loss: 0.0288 - val_loss: 0.0246
Epoch 5/50
2027/2027 [==============================] - 13s 6ms/step - loss: 0.0287 - val_loss: 0.0245
Epoch 6/50
2027/2027 [==============================] - 13s 6ms/step - loss: 0.0287 - val_loss: 0.0246


##### Out of Sample + Total $R^{2}$

In [ ]:
#Use the model weights to predict returns on the validation set
pred = model_2.predict(z_x_test)
in_samp = model_2.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns[2026950:], np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

63343/63343 [==============================] - 117s 2ms/step
Out of Sample R^2: 0.11562461496092236
Total R^2: 0.1640287738784787


##### Predictive R^{2}

In [ ]:
get_6th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[6].output])

lambda_new = get_6th_layer_output_2(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)
get_5th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[5].output])
betas = get_5th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
create_total_r2(returns[2026950:], predicted)

0.004865792703996008

### 3 Hidden Layers

In [ ]:
model_3 = create_three_hidden_layer_network(no_units_first=64, no_units_second=128, no_units_third=32, no_factors=5, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0)
BATCH_SIZE = 1000
EPOCHS = 50
history = model_3.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:2026950],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns[2026950:]),
    callbacks=[earlystop])

Epoch 1/50
2027/2027 [==============================] - 18s 8ms/step - loss: 0.0298 - val_loss: 0.0248
Epoch 2/50
2027/2027 [==============================] - 15s 7ms/step - loss: 0.0291 - val_loss: 0.0245
Epoch 3/50
2027/2027 [==============================] - 15s 8ms/step - loss: 0.0289 - val_loss: 0.0245
Epoch 4/50
2027/2027 [==============================] - 15s 7ms/step - loss: 0.0288 - val_loss: 0.0244
Epoch 5/50
2027/2027 [==============================] - 15s 8ms/step - loss: 0.0287 - val_loss: 0.0245
Epoch 6/50
2027/2027 [==============================] - 15s 7ms/step - loss: 0.0286 - val_loss: 0.0246
Epoch 7/50
2027/2027 [==============================] - 15s 7ms/step - loss: 0.0285 - val_loss: 0.0246


##### Out of Sample + Total $R^{2}$

In [ ]:
pred = model_3.predict(z_x_test)
in_samp = model_3.predict(np.concatenate([z_x_train, z_x_val]))

#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns[2026950:], np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

63343/63343 [==============================] - 112s 2ms/step
Out of Sample R^2: 0.1141766769031084
Total R^2: 0.16257605832847666


##### Predictive $R^{2}$

In [ ]:
get_6th_layer_output_3 = K.function([model_3.layers[0].input],
                                  [model_3.layers[7].output])

lambda_new = get_6th_layer_output_3(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)
get_7th_layer_output_2 = K.function([model_3.layers[0].input],
                                  [model_3.layers[6].output])
betas = get_7th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
create_total_r2(returns[2026950:], predicted)

0.0046259527936586275

## Evaluate for the Case when we restrict only for PCA Observations

In [8]:
def create_total_r2(actual_returns, pred_returns):
    denom = np.sum(actual_returns**2)
    nom = np.sum((actual_returns - pred_returns)**2)
    r_2 = 1 - nom / denom
    return r_2

In [9]:
with open('pca_columns.pkl', 'rb') as f:
    pca_cols = pickle.load(f)

In [30]:
def get_data_to_matrix_pca_names_only(intersection, pca_names, files_from_1970):
    z = np.zeros((1, 70))
    x = np.zeros((1, 70))
    returns = np.array([])
    number_of_obs = {}
    for i, file in enumerate(files_from_1970):
        full_name = 'Data/' + file
        df = pd.read_csv(full_name)
        df.drop(columns=['Unnamed: 0', 'mve0', 'prc', 'SHROUT', 'sic2'], inplace=True)
        df = df[intersection]
        # Onl keep the pca columns
        df = df[df.permno.isin(pca_names)]
        df.dropna(axis=0, thresh=60, inplace=True)
        # df.drop(columns=['RET'], inplace=True)
        df = df.apply(lambda x: (x - x.mean()) / x.std() if (x.name not in ('permno','RET','DATE'))  else x)
        df.fillna(0, inplace=True)
        Z = df.drop(columns=['permno', 'RET', 'DATE'])
        Z = Z.to_numpy()
        X = np.linalg.inv(Z.T @ Z) @ Z.T @ df['RET'].to_numpy()
        X_stacked = np.tile(X, (len(Z), 1))
        ret = df['RET'].to_numpy()
        number_of_obs[file] = (len(ret))
        z = np.vstack((z, Z))
        x = np.vstack((x, X_stacked))
        returns = np.append(returns, ret)
        print(f"File {file} read successfully.")
    z = z[1:]
    x = x[1:]
    return z, x, returns, number_of_obs

In [57]:
def create_total_r2(actual_returns, pred_returns):
    denom = np.sum(actual_returns**2)
    nom = np.sum((actual_returns - pred_returns)**2)
    r_2 = 1 - nom / denom
    return r_2

from keras import backend as K

In [31]:
z_list, x_list,  returns_list, number_of_obs_list = get_data_to_matrix_pca_names_only(intersection, pca_cols, files_from_1970)

File monthly_data/data_for_month_19691231.csv read successfully.
File monthly_data/data_for_month_19700130.csv read successfully.
File monthly_data/data_for_month_19700227.csv read successfully.
File monthly_data/data_for_month_19700331.csv read successfully.
File monthly_data/data_for_month_19700430.csv read successfully.
File monthly_data/data_for_month_19700529.csv read successfully.
File monthly_data/data_for_month_19700630.csv read successfully.
File monthly_data/data_for_month_19700731.csv read successfully.
File monthly_data/data_for_month_19700831.csv read successfully.
File monthly_data/data_for_month_19700930.csv read successfully.
File monthly_data/data_for_month_19701030.csv read successfully.
File monthly_data/data_for_month_19701130.csv read successfully.
File monthly_data/data_for_month_19701231.csv read successfully.
File monthly_data/data_for_month_19710129.csv read successfully.
File monthly_data/data_for_month_19710226.csv read successfully.
File monthly_data/data_fo

In [41]:
# with open('characters_pca.pkl', 'wb') as f:
#     pickle.dump(z_list, f)
# with open('factors_pca.pkl', 'wb') as f:
#     pickle.dump(x_list, f)
# with open('returns_pca.pkl', 'wb') as f:
#     pickle.dump(returns_list, f)
# with open('number_of_obs_pca.pkl', 'wb') as f:
#     pickle.dump(number_of_obs_list, f)

In [40]:
number_of_obs

{'monthly_data/data_for_month_19691231.csv': 1441,
 'monthly_data/data_for_month_19700130.csv': 1447,
 'monthly_data/data_for_month_19700227.csv': 1452,
 'monthly_data/data_for_month_19700331.csv': 1461,
 'monthly_data/data_for_month_19700430.csv': 1476,
 'monthly_data/data_for_month_19700529.csv': 1484,
 'monthly_data/data_for_month_19700630.csv': 1488,
 'monthly_data/data_for_month_19700731.csv': 1608,
 'monthly_data/data_for_month_19700831.csv': 1614,
 'monthly_data/data_for_month_19700930.csv': 1617,
 'monthly_data/data_for_month_19701030.csv': 1623,
 'monthly_data/data_for_month_19701130.csv': 1632,
 'monthly_data/data_for_month_19701231.csv': 1628,
 'monthly_data/data_for_month_19710129.csv': 1635,
 'monthly_data/data_for_month_19710226.csv': 1643,
 'monthly_data/data_for_month_19710331.csv': 1648,
 'monthly_data/data_for_month_19710430.csv': 1662,
 'monthly_data/data_for_month_19710528.csv': 1668,
 'monthly_data/data_for_month_19710630.csv': 1667,
 'monthly_data/data_for_month_1

In [52]:
z_list.shape

(1138634, 70)

In [49]:
sum(list(number_of_obs_list.values())[:421])

519707

In [50]:
sum(list(number_of_obs_list.values())[:481])

749032

In [53]:
with open('characters_pca.pkl', 'rb') as f:
    Z = pickle.load(f)
with open('factors_pca.pkl', 'rb') as f:
    X = pickle.load(f)
with open('returns_pca.pkl', 'rb') as f:
    returns = pickle.load(f)
with open('number_of_obs_pca.pkl', 'rb') as f:
    number_of_obs = pickle.load(f)

In [54]:
z_test = Z[749032:]
x_test = X[749032:]
returns_test = returns[749032:]
z_x_test = np.hstack((z_test, x_test))

z_x_train = np.concatenate((Z[:519707], X[:519707]), axis=1)
z_x_val = np.concatenate((Z[519707:749032], X[519707:749032]), axis=1)

# 3 Factors

### One Hidden Layer

In [56]:
model = create_one_hidden_layer_mod(no_units_activ=16, no_factors=3, activ='relu', l_rate=0.002,
                                     inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                    kernel_init='he_uniform', optimizer=keras.optimizers.Adam)
BATCH_SIZE = 1000
EPOCHS = 50
history = model.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:749032],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_3/strided_slice:0', description="created by layer 'tf.__operators__.getitem_3'")
Epoch 1/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0228 - val_loss: 0.0180
Epoch 2/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0210 - val_loss: 0.0179
Epoch 3/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0209 - val_loss: 0.0179
Epoch 4/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0208 - val_loss: 0.0179
Epoch 5/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0208 - val_loss: 0.0179
Epoch 6/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0207 - val_loss: 0.0179
Epoch 7/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0207 - val_loss: 0.0179
Epoch 8/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0207 - val_loss: 0.0179
Epoch 9

In [58]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_test)
in_samp = model.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

23408/23408 [==============================] - 13s 536us/step
Out of Sample R^2: 0.13307985210577045
Total R^2: 0.17650151063871866


In [59]:
from keras import backend as K

get_5th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
lambda_new = get_5th_layer_output([z_x_train])[0]
lambda_facts = np.mean(lambda_new, axis=0)

# Get the output of the 5th layer
get_4th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[4].output])
betas = get_4th_layer_output(z_x_test)[0]

In [60]:
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)

In [62]:
predicted_r_2 = create_total_r2(returns_test, predicted)
print(f"Predicted R^2: {predicted_r_2}")

Predicted R^2: -0.004658250038879874


##### Two Hidden Layers

In [65]:
model_2 = create_two_hidden_layer_network(no_units_first=64, no_units_second=32, no_factors=3, activ='relu', l_rate=0.002,
                                           inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                           kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_2.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:749032],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

Epoch 1/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0221 - val_loss: 0.0182
Epoch 2/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0211 - val_loss: 0.0181
Epoch 3/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0209 - val_loss: 0.0180
Epoch 4/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0207 - val_loss: 0.0180
Epoch 5/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0206 - val_loss: 0.0181
Epoch 6/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0205 - val_loss: 0.0180
Epoch 7/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0205 - val_loss: 0.0181
Epoch 8/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0204 - val_loss: 0.0181
Epoch 9/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0203 - val_loss: 0.0181


In [68]:
#Use the model weights to predict returns on the validation set
pred = model_2.predict(z_x_test)
in_samp = model_2.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[6].output])

lambda_new = get_6th_layer_output_2(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)
get_5th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[5].output])
betas = get_5th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f" Predictive R^2: {create_total_r2(returns_test, predicted)}")

23408/23408 [==============================] - 14s 577us/step
Out of Sample R^2: 0.12237729958348809
Total R^2: 0.18598202865237745
 Predictive R^2: 0.0018282305884290695


##### Three Hidden Layers

In [69]:
model_3 = create_three_hidden_layer_network(no_units_first=64, no_units_second=128, no_units_third=32, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0,
                                            reg_type=keras.regularizers.L2, kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_3.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:749032],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test,returns_test),
    callbacks=[earlystop])

Epoch 1/50
750/750 [==============================] - 3s 3ms/step - loss: 0.0221 - val_loss: 0.0181
Epoch 2/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0212 - val_loss: 0.0182
Epoch 3/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0210 - val_loss: 0.0183
Epoch 4/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0208 - val_loss: 0.0181
Epoch 5/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0206 - val_loss: 0.0180
Epoch 6/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0204 - val_loss: 0.0181
Epoch 7/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0204 - val_loss: 0.0180
Epoch 8/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0203 - val_loss: 0.0182
Epoch 9/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0202 - val_loss: 0.0180
Epoch 10/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0201 - val_loss: 0.0180

In [70]:
pred = model_3.predict(z_x_test)
in_samp = model_3.predict(np.concatenate([z_x_train, z_x_val]))


#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_3 = K.function([model_3.layers[0].input],
                                  [model_3.layers[7].output])

lambda_new = get_6th_layer_output_3(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)

get_7th_layer_output_2 = K.function([model_3.layers[0].input],
                                  [model_3.layers[6].output])
betas = get_7th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f"Predictiv R^2: {create_total_r2(returns_test, predicted)}")

23408/23408 [==============================] - 13s 561us/step
Out of Sample R^2: 0.1254760225509136
Total R^2: 0.2049515182645817
Predictiv R^2: 0.004107272395861505


## 5 Factors

### One Hidden Layer

In [71]:
model = create_one_hidden_layer_mod(no_units_activ=16, no_factors=5, activ='relu', l_rate=0.002,
                                     inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                    kernel_init='he_uniform', optimizer=keras.optimizers.Adam)
BATCH_SIZE = 1000
EPOCHS = 50
history = model.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:749032],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_9/strided_slice:0', description="created by layer 'tf.__operators__.getitem_9'")
Epoch 1/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0221 - val_loss: 0.0179
Epoch 2/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0208 - val_loss: 0.0179
Epoch 3/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0206 - val_loss: 0.0178
Epoch 4/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0206 - val_loss: 0.0179
Epoch 5/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0205 - val_loss: 0.0179
Epoch 6/50
750/750 [==============================] - 1s 1ms/step - loss: 0.0204 - val_loss: 0.0178
Epoch 7/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0204 - val_loss: 0.0179
Epoch 8/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0204 - val_loss: 0.0178
Epoch 9

In [72]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_test)
in_samp = model.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

23408/23408 [==============================] - 12s 528us/step
Out of Sample R^2: 0.13354397392278805
Total R^2: 0.18259116005488152


In [73]:
from keras import backend as K

get_5th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
lambda_new = get_5th_layer_output([z_x_train])[0]
lambda_facts = np.mean(lambda_new, axis=0)

# Get the output of the 5th layer
get_4th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[4].output])
betas = get_4th_layer_output(z_x_test)[0]

In [74]:
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)

In [75]:
predicted_r_2 = create_total_r2(returns_test, predicted)
print(f"Predicted R^2: {predicted_r_2}")

Predicted R^2: -0.012878600061789758


##### Two Hidden Layers

In [76]:
model_2 = create_two_hidden_layer_network(no_units_first=64, no_units_second=32, no_factors=5, activ='relu', l_rate=0.002,
                                           inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                           kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_2.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:749032],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

Epoch 1/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0219 - val_loss: 0.0179
Epoch 2/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0208 - val_loss: 0.0178
Epoch 3/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0205 - val_loss: 0.0180
Epoch 4/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0204 - val_loss: 0.0178
Epoch 5/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0203 - val_loss: 0.0179
Epoch 6/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0202 - val_loss: 0.0178
Epoch 7/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0201 - val_loss: 0.0181
Epoch 8/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0200 - val_loss: 0.0179
Epoch 9/50
750/750 [==============================] - 1s 2ms/step - loss: 0.0200 - val_loss: 0.0179


In [77]:
#Use the model weights to predict returns on the validation set
pred = model_2.predict(z_x_test)
in_samp = model_2.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[6].output])

lambda_new = get_6th_layer_output_2(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)
get_5th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[5].output])
betas = get_5th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f" Predictive R^2: {create_total_r2(returns_test, predicted)}")

23408/23408 [==============================] - 13s 536us/step
Out of Sample R^2: 0.1341085208784515
Total R^2: 0.19925341219324955
 Predictive R^2: 0.004893425112243777


##### Three Hidden Layers

In [78]:
model_3 = create_three_hidden_layer_network(no_units_first=64, no_units_second=128, no_units_third=32, no_factors=5, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0,
                                            reg_type=keras.regularizers.L2, kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_3.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[:749032],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test,returns_test),
    callbacks=[earlystop])

Epoch 1/50
750/750 [==============================] - 3s 3ms/step - loss: 0.0217 - val_loss: 0.0180
Epoch 2/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0207 - val_loss: 0.0179
Epoch 3/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0205 - val_loss: 0.0179
Epoch 4/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0204 - val_loss: 0.0179
Epoch 5/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0202 - val_loss: 0.0181
Epoch 6/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0201 - val_loss: 0.0180


In [79]:
pred = model_3.predict(z_x_test)
in_samp = model_3.predict(np.concatenate([z_x_train, z_x_val]))


#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_3 = K.function([model_3.layers[0].input],
                                  [model_3.layers[7].output])

lambda_new = get_6th_layer_output_3(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)

get_7th_layer_output_2 = K.function([model_3.layers[0].input],
                                  [model_3.layers[6].output])
betas = get_7th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f"Predictiv R^2: {create_total_r2(returns_test, predicted)}")

23408/23408 [==============================] - 12s 526us/step
Out of Sample R^2: 0.12994331245888646
Total R^2: 0.19789018770990574
Predictiv R^2: 0.002127911838298191


## Estimation only Last 10 Years

In [80]:
sum(list(number_of_obs_list.values())[:362])

341690

In [87]:
sum(list(number_of_obs_list.values())[:421])

516341

In [86]:
sum(list(number_of_obs_list.values())[:481])

744997

In [100]:
z_test = Z[744997:]
x_test = X[744997:]
returns_test = returns[744997:]
z_x_test = np.hstack((z_test, x_test))

returns_eval = returns[341690:]
z_x_train = np.concatenate((Z[341690:516341], X[341690:516341]), axis=1)
z_x_val = np.concatenate((Z[516341:744997], X[516341:744997]), axis=1)

# 3 Factors

### One Hidden Layer

In [91]:
model = create_one_hidden_layer_mod(no_units_activ=16, no_factors=3, activ='relu', l_rate=0.002,
                                     inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                    kernel_init='he_uniform', optimizer=keras.optimizers.Adam)
BATCH_SIZE = 1000
EPOCHS = 50
history = model.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[341690:744997],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_17/strided_slice:0', description="created by layer 'tf.__operators__.getitem_17'")
Epoch 1/50


404/404 [==============================] - 1s 3ms/step - loss: 0.0256 - val_loss: 0.0184
Epoch 2/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0242 - val_loss: 0.0185
Epoch 3/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0239 - val_loss: 0.0183
Epoch 4/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0237 - val_loss: 0.0183
Epoch 5/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0236 - val_loss: 0.0183
Epoch 6/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0236 - val_loss: 0.0183
Epoch 7/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0235 - val_loss: 0.0184


In [101]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_test)
in_samp = model.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns_eval, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

12604/12604 [==============================] - 7s 551us/step
Out of Sample R^2: 0.1110539980682621
Total R^2: 0.17789112492975412


In [102]:
from keras import backend as K

get_5th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
lambda_new = get_5th_layer_output([z_x_train])[0]
lambda_facts = np.mean(lambda_new, axis=0)

# Get the output of the 5th layer
get_4th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[4].output])
betas = get_4th_layer_output(z_x_test)[0]

In [103]:
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)

In [104]:
predicted_r_2 = create_total_r2(returns_test, predicted)
print(f"Predicted R^2: {predicted_r_2}")

Predicted R^2: -0.011517637241458267


##### Two Hidden Layers

In [105]:
model_2 = create_two_hidden_layer_network(no_units_first=64, no_units_second=32, no_factors=3, activ='relu', l_rate=0.002,
                                           inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                           kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_2.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[341690:744997],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

Epoch 1/50


404/404 [==============================] - 2s 4ms/step - loss: 0.0255 - val_loss: 0.0184
Epoch 2/50
404/404 [==============================] - 2s 4ms/step - loss: 0.0239 - val_loss: 0.0184
Epoch 3/50
404/404 [==============================] - 1s 4ms/step - loss: 0.0236 - val_loss: 0.0183
Epoch 4/50
404/404 [==============================] - 1s 4ms/step - loss: 0.0234 - val_loss: 0.0184
Epoch 5/50
404/404 [==============================] - 1s 4ms/step - loss: 0.0232 - val_loss: 0.0185
Epoch 6/50
404/404 [==============================] - 1s 4ms/step - loss: 0.0231 - val_loss: 0.0186


In [106]:
#Use the model weights to predict returns on the validation set
pred = model_2.predict(z_x_test)
in_samp = model_2.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns_eval, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[6].output])

lambda_new = get_6th_layer_output_2(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)
get_5th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[5].output])
betas = get_5th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f" Predictive R^2: {create_total_r2(returns_test, predicted)}")

12604/12604 [==============================] - 7s 579us/step
Out of Sample R^2: 0.10124720141521759
Total R^2: 0.18580826891247915
 Predictive R^2: 0.0028726275103817533


##### Three Hidden Layers

In [107]:
model_3 = create_three_hidden_layer_network(no_units_first=64, no_units_second=128, no_units_third=32, no_factors=3, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0,
                                            reg_type=keras.regularizers.L2, kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_3.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[341690:744997],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test,returns_test),
    callbacks=[earlystop])

Epoch 1/50
404/404 [==============================] - 3s 7ms/step - loss: 0.0254 - val_loss: 0.0185
Epoch 2/50
404/404 [==============================] - 3s 7ms/step - loss: 0.0239 - val_loss: 0.0186
Epoch 3/50
404/404 [==============================] - 3s 6ms/step - loss: 0.0236 - val_loss: 0.0187
Epoch 4/50
404/404 [==============================] - 3s 7ms/step - loss: 0.0233 - val_loss: 0.0186


In [108]:
pred = model_3.predict(z_x_test)
in_samp = model_3.predict(np.concatenate([z_x_train, z_x_val]))


#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns_eval, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_3 = K.function([model_3.layers[0].input],
                                  [model_3.layers[7].output])

lambda_new = get_6th_layer_output_3(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)

get_7th_layer_output_2 = K.function([model_3.layers[0].input],
                                  [model_3.layers[6].output])
betas = get_7th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f"Predictiv R^2: {create_total_r2(returns_test, predicted)}")

12604/12604 [==============================] - 7s 587us/step
Out of Sample R^2: 0.10244002262917884
Total R^2: 0.18424791093376625
Predictiv R^2: 0.005821496871401766


## 5 Factors

### One Hidden Layer

In [109]:
model = create_one_hidden_layer_mod(no_units_activ=16, no_factors=5, activ='relu', l_rate=0.002,
                                     inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                    kernel_init='he_uniform', optimizer=keras.optimizers.Adam)
BATCH_SIZE = 1000
EPOCHS = 50
history = model.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[341690:744997],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), name='tf.__operators__.getitem_23/strided_slice:0', description="created by layer 'tf.__operators__.getitem_23'")
Epoch 1/50
404/404 [==============================] - 2s 3ms/step - loss: 0.0258 - val_loss: 0.0184
Epoch 2/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0239 - val_loss: 0.0184
Epoch 3/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0235 - val_loss: 0.0183
Epoch 4/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0234 - val_loss: 0.0183
Epoch 5/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0233 - val_loss: 0.0184
Epoch 6/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0232 - val_loss: 0.0184
Epoch 7/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0232 - val_loss: 0.0184


In [110]:
#Use the model weights to predict returns on the validation set
pred = model.predict(z_x_test)
in_samp = model.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns_eval, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

12604/12604 [==============================] - 7s 558us/step
Out of Sample R^2: 0.11246264162397945
Total R^2: 0.18573563030927287


In [111]:
from keras import backend as K

get_5th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
lambda_new = get_5th_layer_output([z_x_train])[0]
lambda_facts = np.mean(lambda_new, axis=0)

# Get the output of the 5th layer
get_4th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[4].output])
betas = get_4th_layer_output(z_x_test)[0]

In [112]:
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)

In [113]:
predicted_r_2 = create_total_r2(returns_test, predicted)
print(f"Predicted R^2: {predicted_r_2}")

Predicted R^2: -0.008113221259306691


##### Two Hidden Layers

In [114]:
model_2 = create_two_hidden_layer_network(no_units_first=64, no_units_second=32, no_factors=5, activ='relu', l_rate=0.002,
                                           inp_shape=z_x_train.shape[1], reg=0, reg_type=keras.regularizers.L2,
                                           kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_2.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[341690:744997],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test, returns_test),
    callbacks=[earlystop])

Epoch 1/50


404/404 [==============================] - 2s 3ms/step - loss: 0.0252 - val_loss: 0.0185
Epoch 2/50
404/404 [==============================] - 1s 3ms/step - loss: 0.0237 - val_loss: 0.0185
Epoch 3/50
404/404 [==============================] - 1s 3ms/step - loss: 0.0234 - val_loss: 0.0183
Epoch 4/50
404/404 [==============================] - 1s 3ms/step - loss: 0.0231 - val_loss: 0.0184
Epoch 5/50
404/404 [==============================] - 1s 3ms/step - loss: 0.0229 - val_loss: 0.0187
Epoch 6/50
404/404 [==============================] - 1s 3ms/step - loss: 0.0228 - val_loss: 0.0185


In [115]:
#Use the model weights to predict returns on the validation set
pred = model_2.predict(z_x_test)
in_samp = model_2.predict(np.concatenate([z_x_train, z_x_val]))
#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns_eval, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[6].output])

lambda_new = get_6th_layer_output_2(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)
get_5th_layer_output_2 = K.function([model_2.layers[0].input],
                                  [model_2.layers[5].output])
betas = get_5th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f" Predictive R^2: {create_total_r2(returns_test, predicted)}")

12604/12604 [==============================] - 7s 557us/step
Out of Sample R^2: 0.1056840800723744
Total R^2: 0.1931405549807439
 Predictive R^2: 0.0021178369735930236


##### Three Hidden Layers

In [116]:
model_3 = create_three_hidden_layer_network(no_units_first=64, no_units_second=128, no_units_third=32, no_factors=5, activ='relu', l_rate=0.002, inp_shape=z_x_train.shape[1], reg=0,
                                            reg_type=keras.regularizers.L2, kernel_init='he_uniform')
BATCH_SIZE = 1000
EPOCHS = 50
history = model_3.fit(
    x=np.concatenate([z_x_train, z_x_val]),
    y=returns[341690:744997],
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose='auto',
    validation_data=(z_x_test,returns_test),
    callbacks=[earlystop])

Epoch 1/50
404/404 [==============================] - 3s 5ms/step - loss: 0.0250 - val_loss: 0.0184
Epoch 2/50
404/404 [==============================] - 2s 5ms/step - loss: 0.0237 - val_loss: 0.0186
Epoch 3/50
404/404 [==============================] - 2s 4ms/step - loss: 0.0233 - val_loss: 0.0186
Epoch 4/50
404/404 [==============================] - 2s 4ms/step - loss: 0.0230 - val_loss: 0.0185


In [117]:
pred = model_3.predict(z_x_test)
in_samp = model_3.predict(np.concatenate([z_x_train, z_x_val]))


#Calculate the R-squared of the model
print(f"Out of Sample R^2: {create_total_r2(returns_test, np.concatenate(pred))}")
print(f"Total R^2: {create_total_r2(returns_eval, np.concatenate([np.concatenate(in_samp), np.concatenate(pred)]))}")

get_6th_layer_output_3 = K.function([model_3.layers[0].input],
                                  [model_3.layers[7].output])

lambda_new = get_6th_layer_output_3(z_x_test)[0]
lambda_facts = np.mean(lambda_new, axis=0)

get_7th_layer_output_2 = K.function([model_3.layers[0].input],
                                  [model_3.layers[6].output])
betas = get_7th_layer_output_2(z_x_test)[0]
predicted = betas * np.tile(lambda_facts,(z_x_test[:,70:].shape[0],1))
predicted = np.sum(predicted,axis=1)
print(f"Predictiv R^2: {create_total_r2(returns_test, predicted)}")

12604/12604 [==============================] - 7s 581us/step
Out of Sample R^2: 0.10698218439072571
Total R^2: 0.19127113167868892
Predictiv R^2: -6.334219229064963e-05
